<a id="top"></a>
# MAST API Tutorial - Astroquery API

## Introduction and Goals:
This is a beginner tutorial on accessing the MAST database using the Astroquery API. By the end of this tutorial, you will:

* Understand how to search for observations hosted on the MAST archive
* Download data products corresponding to your observations of interest
* Create a visual display of the downloaded data

## Table of Contents

* [Imports](#Imports)


* [Three ways to search for MAST observations](#Three-Ways-to-Search-for-MAST-Observations)
    - [By Region](#1.-By-Region)
    - [By Object Name](#2.-By-Object-Name)
    - [By Criteria](#crit)
    
    
* [Getting Associated Data Products](#Getting-Associated-Data-Products)
    - [Performing a Product Query](#Performing-a-Product-Query)
    - [Filtering Data Products](#Filtering-the-Data-Products)
    
    
* [Downloading Products](#Downloading-Products)


* [Displaying Data](#Displaying-Data)


* [Further Reading](#Further-Reading)

## Imports

Let's start by importing the packages we need for this notebook.

* `astroquery.mast` to access the MAST API
* `astropy` for coordinate objects and opening the data
* `matplotlib` for plotting the data

In [ ]:
from astroquery.mast import Observations

from astropy.io import fits
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
from matplotlib.colors import SymLogNorm
%matplotlib inline

## Three Ways to Search for MAST Observations

### 1. By Region
You can use `query_region` to find objects based on coordinates and radius.

The coordinates can be given as a string or `astropy.coordinates` object, and the radius as a string or float object. If no radius is specified, the default is 0.2 degrees.

In [ ]:
# This will give a warning that the coordinates are being interpreted as an ICRS coordinate provided in degrees
obsByRegion = Observations.query_region("322.49324 12.16683")

If we want to avoid the the above warning we can create a `coordinates` object. In addition, let's add a radius of 5 arcseconds to this search.

In [ ]:
# Set up our coordinates object
coord = SkyCoord(322.49324, 12.16683, unit='deg')

# Same search as above, now with a radius of 5 arcseconds
obsByRegion2 = Observations.query_region(coord, radius='5s')

In [ ]:
# How many results for each search?
print("Results in region 1:",len(obsByRegion),
      '\nResults in region 2:', len(obsByRegion2))

We get fewer results in the second search, but that's expected! The default radius value is 0.2 degrees, or 720 arcseconds, and we performed a search with radius 5 arcseconds.

Now let's preview the first five results of the first search.

In [ ]:
obsByRegion[:5].show_in_notebook()

The columns of the above table (there are many!) correspond to searchable fields in the MAST database. You can find the full list of criteria [here](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html) and an example of performing a search by criteria [here in this notebook](#crit).

### 2. By Object Name

The `query.object` method allows you to search based on object name and optional radius argument. 

The object name is resolved to coordinates by a call to the [Simbad](https://simbad.unistra.fr/simbad/) and [NED](https://ned.ipac.caltech.edu/Documents/Overview) archives. 

In [ ]:
obsByName = Observations.query_object("M51",radius=".005 deg")

In [ ]:
print("Number of results:",len(obsByName))
print(obsByName[:10])

For some special catalogs, like those used with Kepler, K2, and TESS, astroquery performs a direct lookup using the MAST catalog. It is important to include both the **_catalog identifier_** and number when searching one of these datasets; for example, to query the TESS Input catalog you would use "TIC 261136679", rather than "261136679".

In [ ]:
obsByTessName = Observations.query_object("TIC 261136679", radius="1s")

In [ ]:
obsByTessName.show_in_notebook()

### 3. By Other Criteria (with or without name/region) <a id="crit"></a>

To search for observations based on additonal parameters, you can use `query_criteria`. In a sense, this is a more powerful version of the tools above, as you can still search by coordinates and objectname; however, you can inculde additional desired criteria. You may also run the search without specifying a name or coordinates.

To perform the search, give your critera as keyword arguments. Valid criteria and their descriptions are listed [here](https://mast.stsci.edu/api/v0/_c_a_o_mfields.html). Some relevant examples are: "filters", "t_exptime" (exposure time), instrument_name, "provenance_name", and "sequence_number". 

Let's look at an search for TESS campaign 9 data.

In [ ]:
obsByCriteria = Observations.query_criteria(obs_collection=["TESS"], sequence_number=9,
                                            t_exptime=[1400,1500])

In [ ]:
obsByCriteria.show_in_notebook(display_length=5)

There's no limit on the number of filters you can apply in a search. It may be an interesting exercise for the reader to go through the example below and figure out what exactly we're searching for.

In [ ]:
# Make sure to run this cell, as the data is used in the sections that follow

exByCriteria = Observations.query_criteria(obs_collection=["HLA"], s_dec=[50,60], 
                                            calib_level=[3], proposal_pi="Mould*", 
                                            dataproduct_type="IMAGE", t_max=[49800,49820])

In [ ]:
exByCriteria.show_in_notebook()

## Getting Associated Data Products

### Performing a Product Query


Each observation returned from a MAST query can have one or more associated data products. When you input observations or observation ids (“obs_id”), `get_product_list` will return a table containing the associated data products. The product fields are documented [here](https://mast.stsci.edu/api/v0/_productsfields.html).

Since we already have a list of observations, we can use that as the starting point for our query. To keep it simple, let's look at only the last observation from our search above.

In [ ]:
# Let's select a small subset from our critera search above
newObsList = exByCriteria[-1:]

# Now we get the list of products associated with that observation
dataProducts = Observations.get_product_list(newObsList)

In [ ]:
dataProducts.show_in_notebook(display_length=5)

### Filtering the Data Products

After the data products have been retrieved, you can use `filter_products` to download only data products that meet your given criteria. Available filters are listed [here](https://mast.stsci.edu/api/v0/_productsfields.html). Some examples are: “mrp_only” (Minimum Recommended Products) and “extension” (file extension).

A note on filtering: each listed filter is joined with an AND, but each option within that filter is joined with an OR. For example, the search below will return any products that are science type and have a calibration level of 2 or 3.

In [ ]:
scienceProducts = Observations.filter_products(dataProducts, productType=["SCIENCE"],
                                               calib_level=[2,3], mrp_only=False)

In [ ]:
scienceProducts.show_in_notebook(display_length=5)

## Downloading Products
Passing a table of products (like the one above) to `download_products` will download the entire table. You can also pass a list of Observation IDs (obs_id) if you know them.

`download_products` also allows you to filter data as you request the download. In the example below, we will only download the drizzled files (drz.fits).

Products will by default be downloaded into the current working directory, in a subdirectory called "mastDownload."<br>
The full local file paths will have the form "mastDownload/Mission/Observation ID/file."

In [ ]:
# This is the filtered download of the scienceProducts table
manifest = Observations.download_products(scienceProducts, extension=("drz.fits"))

# Uncomment below for "plain" download of the scienceProducts table
# manifest = Observations.download_products(scienceProducts)

Note: `download_products` includes caching by default. If you have downloaded the files before, they will not be downloaded again unless caching is turned off. This may cause issues if the data is updated and the filename remains the same!

In [ ]:
manifest.show_in_notebook()

The manifest returns useful information about the status of the files. You can find the local path, along with a status. This will either be COMPLETE, SKIPPED, or ERROR. If the status is ERROR, there will be additional information in the 'Message' column. The URL field includes a link to directly download the data.

## Displaying Data

At this point the data is ready for analysis, and we are done using `mast.astroquery`. <br>
Below we take a look at the data files using `astropy` and `matplotlib`.

In [ ]:
file1 = fits.open(manifest['Local Path'][0])
file2 = fits.open(manifest['Local Path'][1])

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2)
f.set_figheight(5)
f.set_figwidth(12)
ax1.imshow(file1[0].data, cmap="inferno", norm=SymLogNorm(linthresh=0.03,vmin=0, vmax=1.5))
ax2.imshow(file2['SCI'].data, cmap="inferno", norm=SymLogNorm(linthresh=0.03,vmin=-0.01, vmax=1.5))

## Further Reading
Full documentation on **`astroquery.mast`** can be found [here](https://astroquery.readthedocs.io/en/latest/mast/mast.html).

## About this Notebook
For additonal questions, comments, or feedback, please email `archive@stsci.edu`. 

**Authors:** Thomas Dutkiewicz, Scott Fleming <br>
**Keywords:** MAST, astroquery <br>
**Latest update** July 2022 <br>
**Next Review:** Jan 2023

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 